# Tests on dinov2 embeddings

## Imports

In [1]:
import os
import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Checking torch cache dir
Warning : make sure to set torch hub cache dir to reuse downloade

In [3]:
torch.hub.get_dir()

'/linkhome/idris/genidr/ssos023/.cache/torch/hub'

## Setting torch cache dir for download

In [7]:
# Assuming $ALL_CCFRWORK is an environment variable that stores the desired path
hub_dir = os.path.join('/gpfswork/rech/ads/commun','models')

if hub_dir is not None:
    torch.hub.set_dir(hub_dir)
else:
    print("Environment variable 'ALL_CCFRWORK' is not set.")


## Download dinov2 weights

## Loading a dinov2

In [8]:
dinov2_vitg14_reg = torch.hub.load(repo_or_dir='facebookresearch/dinov2', model='dinov2_vitg14_reg').to(device)

Using cache found in /gpfswork/rech/ads/commun/models/facebookresearch_dinov2_main
/gpfswork/rech/ads/commun/models/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/gpfswork/rech/ads/commun/models/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/gpfswork/rech/ads/commun/models/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


In [14]:
random_image = torch.randn(1, 3, 224, 224).to(device)  # Batch size of 1

output = dinov2_vitg14_reg(random_image)

In [18]:
output.shape

torch.Size([1, 1536])

In [17]:
dinov2_vitg14_reg

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1536, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-39): 40 x NestedTensorBlock(
      (norm1): LayerNorm((1536,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=1536, out_features=4608, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1536, out_features=1536, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((1536,), eps=1e-06, elementwise_affine=True)
      (mlp): SwiGLUFFNFused(
        (w12): Linear(in_features=1536, out_features=8192, bias=True)
        (w3): Linear(in_features=4096, out_features=1536, bias=True)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (norm): LayerNorm((1536,), eps=1e-06, elementwise_affine=True)
  (head